In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# --- ধাপ ১: ডামি ডেটাসেট তৈরি করো (বাস্তবে তুমি একটি বড় ডেটাসেট লোড করবে) ---
# এই ডেটাসেটটি ছোট আকারের, শুধুমাত্র উদাহরণের জন্য।
# বাস্তব ডেটাসেট: যেমন IMDB মুভি রিভিউ ডেটাসেট (অনেক বড়)
data = {
    'review': [
        "This movie was absolutely fantastic! A must-watch.",
        "Terrible acting and a boring plot. Don't waste your time.",
        "A decent film, but nothing extraordinary.",
        "I loved every minute of it, truly a masterpiece.",
        "Worst movie ever, completely unwatchable.",
        "Surprisingly good, exceeded my expectations.",
        "Mediocre at best, very predictable.",
        "Highly recommended, great story and visuals.",
        "So disappointing, felt like a chore to finish.",
        "Enjoyed it thoroughly, great performances.",
        "It was okay, not bad, not great.",
        "Absolutely brilliant, a cinematic triumph.",
        "What a mess, confusing and poorly executed.",
        "Very engaging and thought-provoking.",
        "Couldn't stand it, walked out halfway."
    ],
    'sentiment': [
        'positive', 'negative', 'neutral', 'positive', 'negative',
        'positive', 'negative', 'positive', 'negative', 'positive',
        'neutral', 'positive', 'negative', 'positive', 'negative'
    ]
}
df = pd.DataFrame(data)

# 'neutral' রিভিউগুলোকে বাদ দিচ্ছি বা পজিটিভ/নেগেটিভে অ্যাসাইন করছি
# সেন্টিমেন্ট অ্যানালাইসিসের জন্য সাধারণত বাইনারি ক্লাসিফিকেশন (পজিটিভ/নেগেটিভ) করা হয়।
# এখানে 'neutral' রিভিউগুলোকে বাদ দিচ্ছি বোঝার সুবিধার জন্য।
df = df[df['sentiment'] != 'neutral']

# সেন্টিমেন্ট লেবেলগুলোকে সংখ্যায় রূপান্তর করো: 'positive' -> 1, 'negative' -> 0
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})

print("প্রসেস করা ডেটাসেটের প্রথম ৫টি সারি:")
print(df.head())
print(f"\nমোট রিভিউ সংখ্যা: {len(df)}")

# ফিচার (X) এবং টার্গেট (y) আলাদা করো
X_text = df['review'].values
y = df['sentiment'].values

# --- ধাপ ২: টেক্সট ডেটা প্রিপারেশন (Tokenization and Padding) ---

# টোকেনাইজার তৈরি করো: এটি শব্দগুলোকে সংখ্যায় রূপান্তর করবে
# num_words: কতগুলো সবচেয়ে বেশি ব্যবহৃত শব্দ বিবেচনা করা হবে
# oov_token: অজানা শব্দগুলোর জন্য একটি টোকেন (Out-Of-Vocabulary)
vocab_size = 1000 # তুমি ডেটাসেটের আকারের উপর নির্ভর করে এটি বাড়াতে পারো
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<unk>")
tokenizer.fit_on_texts(X_text) # টেক্সট ডেটার উপর টোকেনাইজারকে ফিট করো

# টেক্সটগুলোকে সংখ্যার সিকোয়েন্সে রূপান্তর করো
X_sequences = tokenizer.texts_to_sequences(X_text)

# সিকোয়েন্সগুলোর দৈর্ঘ্য সমান করো (প্যাডিং)
# maxlen: প্রতিটি সিকোয়েন্সের সর্বোচ্চ দৈর্ঘ্য
# padding='post': যদি সিকোয়েন্স ছোট হয়, তাহলে শেষে শূন্য যোগ করা হবে
# truncating='post': যদি সিকোয়েন্স বড় হয়, তাহলে শেষে কেটে ফেলা হবে
maxlen = 50 # প্রতিটি রিভিউয়ের সর্বোচ্চ শব্দ সংখ্যা
X_padded = pad_sequences(X_sequences, maxlen=maxlen, padding='post', truncating='post')

print(f"\nপ্রথম রিভিউয়ের মূল টেক্সট: {X_text[0]}")
print(f"প্রথম রিভিউয়ের টোকেনাইজড সিকোয়েন্স: {X_sequences[0]}")
print(f"প্রথম রিভিউয়ের প্যাডেড সিকোয়েন্স (maxlen={maxlen}): {X_padded[0]}")

# --- ধাপ ৩: ডেটা সেটকে ট্রেনিং এবং টেস্টিং সেটে ভাগ করো ---
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

print(f"\nট্রেনিং ডেটার আকার: {X_train.shape}")
print(f"টেস্টিং ডেটার আকার: {X_test.shape}")

# --- ধাপ ৪: ডিপ লার্নিং মডেল তৈরি করো (LSTM ব্যবহার করে) ---
embedding_dim = 100 # প্রতিটি শব্দের জন্য এমবেডিং ভেক্টরের মাত্রা

model = Sequential()
# Embedding Layer: শব্দ ইনডেক্সগুলোকে ঘন ভেক্টরে রূপান্তর করে
# input_dim: শব্দভাণ্ডারের আকার (vocab_size)
# output_dim: এমবেডিং ভেক্টরের মাত্রা
# input_length: প্রতিটি ইনপুট সিকোয়েন্সের দৈর্ঘ্য (maxlen)
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=maxlen))

# LSTM Layer: সিকোয়েন্স ডেটা প্রসেস করার জন্য
# 128: LSTM ইউনিটের সংখ্যা (তুমি এটি পরিবর্তন করতে পারো)
model.add(LSTM(128))

# Dropout Layer: ওভারফিটিং কমানোর জন্য (কিছু নিউরনকে র্যান্ডমলি বন্ধ করে দেয়)
model.add(Dropout(0.5))

# Dense Output Layer: বাইনারি ক্লাসিফিকেশনের জন্য 1টি নিউরন
# activation='sigmoid': আউটপুটকে 0 থেকে 1 এর মধ্যে সম্ভাবনায় রূপান্তর করে
model.add(Dense(1, activation='sigmoid'))

# মডেলের সারসংক্ষেপ দেখাও
model.summary()

# --- ধাপ ৫: মডেল কম্পাইল করো ---
# optimizer: মডেলকে কিভাবে শিখতে হবে তা নিয়ন্ত্রণ করে (Adam একটি জনপ্রিয় অপ্টিমাইজার)
# loss: মডেলের প্রেডিকশন এবং আসল লেবেলের মধ্যে পার্থক্য পরিমাপ করে (বাইনারি ক্লাসিফিকেশনের জন্য binary_crossentropy)
# metrics: ট্রেনিং এবং মূল্যায়নের সময় মডেলের পারফরম্যান্স পরিমাপ করার জন্য (accuracy)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# --- ধাপ ৬: মডেল ট্রেনিং করো ---
# EarlyStopping: যদি মডেলের পারফরম্যান্স (validation loss) নির্দিষ্ট সংখ্যক এপক ধরে উন্নত না হয়, তাহলে ট্রেনিং বন্ধ করে দেবে।
# patience: কতগুলো এপক ধরে উন্নতি না হলে ট্রেনিং বন্ধ হবে
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

print("\nমডেল ট্রেনিং শুরু হচ্ছে...")
history = model.fit(
    X_train, y_train,
    epochs=50, # তুমি আরও বেশি এপক দিতে পারো, EarlyStopping এটি নিয়ন্ত্রণ করবে
    batch_size=32, # একবারে কতগুলো স্যাম্পল মডেল দেখবে
    validation_split=0.2, # ট্রেনিং ডেটার একটি অংশ ভ্যালিডেশনের জন্য ব্যবহার করা হবে
    callbacks=[early_stopping], # EarlyStopping কলব্যাক যোগ করা হয়েছে
    verbose=1
)
print("মডেল ট্রেনিং সম্পন্ন হয়েছে!")

# --- ধাপ ৭: মডেল মূল্যায়ন করো ---
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\nটেস্টিং ডেটার উপর মডেলের নির্ভুলতা (Accuracy): {accuracy:.4f}")
print(f"টেস্টিং ডেটার উপর মডেলের লস (Loss): {loss:.4f}")

# --- ধাপ ৮: নতুন রিভিউয়ের জন্য প্রেডিকশন করো ---
new_reviews = [
    "This movie was truly amazing and captivating. Highly recommend!",
    "Absolutely terrible, a complete waste of money and time. Avoid it.",
    "It was an average film, nothing special.",
    "The plot was confusing and the acting was wooden."
]

# নতুন রিভিউগুলোকে প্রসেস করো
new_sequences = tokenizer.texts_to_sequences(new_reviews)
new_padded = pad_sequences(new_sequences, maxlen=maxlen, padding='post', truncating='post')

# প্রেডিকশন করো
predictions = model.predict(new_padded)

print("\nনতুন রিভিউয়ের প্রেডিকশন:")
for i, review in enumerate(new_reviews):
    sentiment_score = predictions[i][0]
    sentiment_label = "Positive" if sentiment_score >= 0.5 else "Negative"
    print(f"রিভিউ: '{review}'")
    print(f"  অনুভূতির স্কোর: {sentiment_score:.4f} -> অনুমিত অনুভূতি: {sentiment_label}")


ImportError: Traceback (most recent call last):
  File "c:\Users\nifad\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.